# User Based Collaborative Filtering 

In [ ]:
import os
import pandas as pd
import requests
import json
import warnings
import sys
sys.path.append('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data_collection/notiAPI')

from auth import NotiServerAuth
from api import NotiServerRequest

from dataclasses import dataclass, field
from typing import Optional, List
from urllib.parse import urlparse, parse_qs
from sklearn.metrics.pairwise import cosine_similarity

from pprint import pprint
warnings.filterwarnings(action='ignore')

---

## 데이터 로드

In [ ]:
# 최근 10일치 데이터, spec 상품 id 추출

df = pd.read_csv('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data/spec.csv') 
df = df[['user', 'item']]
df

---

## 데이터 전처리

In [ ]:
# 각 유저별로 인터렉션 횟수 계산
interaction_counts = df['user'].value_counts()
interaction_counts

In [ ]:
# 15,547 인터렉션 유저 이상치 데이터 제거 
df = df[df['user'] != '439e50a4c439c71ab0eb66ce03e542ca']

In [ ]:
# 인터렉션 횟수가 5회 이상인 유저들의 데이터 추출
# 5회 이상인 유저가 2290 으로 매우 적음
users_with_5_or_more_interactions = interaction_counts[interaction_counts >= 5].index
df = df[df['user'].isin(users_with_5_or_more_interactions)]
df

---

## User based Collaborative Filtering (UBCF)

In [ ]:
# 사용자와 아이템 간의 상호작용 횟수를 계산하고, 상호작용이 없는 경우 0으로 채웁니다.
user_item_matrix = df.pivot_table(index='user', columns='item', aggfunc=len, fill_value=0)
user_item_matrix

In [ ]:
# 사용자 간의 유사도 행렬을 생성, 행렬의 인덱스와 컬럼은 사용자 ID

user_similarity = cosine_similarity(user_item_matrix) 
print('user_similarity shape: ', user_similarity.shape)
user_similarity

In [ ]:
# 주어진 사용자와 유사한 사용자들의 목록과 그들과의 유사도를 반환

def UBCF(df: pd.DataFrame,
         user: int,
         topn: Optional[int]=None) -> pd.DataFrame:

    topn=11 if topn is None else topn+1
    
    if user in df['user'].values:
        user = str(user)
        
        user_item_matrix = df.pivot_table(index='user', columns='item', aggfunc=len, fill_value=0)

        user_similarity_df = pd.DataFrame(cosine_similarity(user_item_matrix), index=user_item_matrix.index.astype(str), columns=user_item_matrix.index.astype(str))
        
        sim_user_df = user_similarity_df[user].sort_values(ascending=False).reset_index(drop=False).rename(columns={'index':'user', user:'similarity'})
        print('입력한 사용자 id: ',user)
        return(sim_user_df[1:topn]) #  0번째 인덱스가 대상 사용자 자신
    else:
        print('사용자 id를 다시 확인해주세요')

In [ ]:
# 유사 유저 5명 id, similarity

UBCF(df, 'e44ba8873d8672b0b427a482996a2de0', 5)

In [ ]:
# 주어진 사용자와 가장 유사한 상위 3명의 사용자를 찾아내고, 이들이 상호작용한 아이템 중에서 입력 사용자가 아직 상호작용하지 않은 아이템을 추천 목록으로 반환

def Recommend_UBCF(df: pd.DataFrame,
                        user: str) -> list:
    
    print('입력 유저 ids ', user)
    
    user_item_matrix = df.pivot_table(index='user', columns='item', aggfunc=len, fill_value=0)

    user_similarity_df = pd.DataFrame(cosine_similarity(user_item_matrix), index=user_item_matrix.index.astype(str), columns=user_item_matrix.index.astype(str))

    inter_user = user_similarity_df[str(user)].sort_values(ascending=False)[1:4].index # 입력 사용자(user)와 가장 유사한 상위 3명의 사용자, 자기 자신 제외
    
    print('유사 유저 ids: ', list(user_similarity_df[str(user)].sort_values(ascending=False)[1:4].index))
    
    # 유사 사용자 각각의 상위 10개 아이템(가장 최근 또는 가장 많이 상호작용한 아이템을 가정)을 추출하고, 
    # 이 중에서 입력 사용자가 아직 상호작용하지 않은 아이템들을 추천 목록으로 선정합니다. 
    # 여기서 set 연산을 사용하여 중복을 제거하고, | 연산자로 합집합을 구한 후, - 연산자로 차집합을 구하여 이미 상호작용한 아이템을 제외합니다.
    
    items = list((set(df[df['user']==inter_user[0]]['item'].value_counts().head(10).index)|
                 set(df[df['user']==inter_user[1]]['item'].value_counts().head(10).index)|
                 set(df[df['user']==inter_user[2]]['item'].value_counts().head(10).index)) - set(df[df['user']==user]['item'].values))
    
    return items


In [ ]:
# e44ba8873d8672b0b427a482996a2de0 와 유사한 유저들이 선호하는 아이템 추천

result = Recommend_UBCF(df, 'e44ba8873d8672b0b427a482996a2de0')
result

In [ ]:
# 각 아이디에 대해 제품 정보를 가져와서 상품명 출력

for item_id in result:
    res = NotiServerRequest.bulk_product_info(ids=[item_id], pivs=[])
    product_info = res.json().get('data', {}).get('products', [])
    if product_info:
        title = product_info[0]['title'].split(',')[0]
        # print("아이템 ID:", item_id)
        print("추천 :", title)
    else:
        # print("아이템 ID:", item_id)
        print("해당 아이템에 대한 정보를 찾을 수 없습니다.")


In [ ]:
# 입력 유저 상호작용 아이템 출력

user_item = df[df['user']=='e44ba8873d8672b0b427a482996a2de0']['item'].tolist()

print('입력 유저 인터렉션 아이템 : ')
# 각 아이디에 대해 제품 정보를 가져와서 타이틀을 출력
tmp = []

for item_id in user_item:
    res = NotiServerRequest.bulk_product_info(ids=[item_id], pivs=[])
    product_info = res.json().get('data', {}).get('products', [])
    if product_info:
        title = product_info[0]['title'].split(',')[0]
        # print(title)
        tmp.append(title)
    else:
        print("해당 아이템에 대한 정보를 찾을 수 없습니다.")

set(tmp)